# Úkol č. 2 - předzpracování dat a binární klasifikace (do 10. listopadu)

  * Cílem tohoto úkolu je vyzkoušet si naučit prediktivní model pro binární klasifikaci.
  * Budete se muset vypořádat s příznaky, které jsou různých typů a které bude třeba nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na trénovací, testovací a případně i validační množinu (preferujeme ale použití cross-validation).
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací/validační množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí testovací množiny. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor(y) s predikcemi pro vyhodnocovací data.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. **První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [2]:
data = pd.read_csv('data.csv')
# display(data.shape)
# display(data.head())
# display(data.describe())
# display(data.nunique())
# display(data.info())

**Zobrazim si informace o sloupcich, ve kterych chybi data**

In [3]:
data.loc[:,data.isnull().sum() > 0].isnull().sum()

age          201
fare           1
cabin        773
embarked       2
home.dest    435
dtype: int64

* Pri pruzkumu dat jsem dosel k nazoru, ze nektera chybejici data si mohu dovolit doplnit na zaklade dat dostupnych pomoci jednoduchych technik
    * **Fare** chybi jen v jednom radku => doplnim hodnotou prumeru
    * **Embarked** chybi ve dvou radcich, doplnim informaci nejcastejsim hodnotou

In [4]:
# doplneni radku s chybejici fare hodnotou
indexOfMissingFareRow = data.loc[data.fare.isnull()].index
data.loc[indexOfMissingFareRow, 'fare'] = data.loc[data.fare.notnull()]['fare'].mean()

# moznost, kde misto prumeru chybejici data nahradim medianem
# imputer = SimpleImputer(missing_values=np.nan, strategy='median')
# data[['fare']] = imputer.fit_transform(data[['fare']])

# doplneni radku s chybejici embarked hodnotou
mostCommonEmbarkedValue = data['embarked'].value_counts().index[0]
for i, row in data.loc[data.embarked.isnull()].iterrows():
    data.at[i, 'embarked'] = mostCommonEmbarkedValue

* **cabin** sloupec sice chybi ve vetsine pripadu, ale docetl jsem se, ze informace o kabinach byly dochovany u vsech lidi, kteri cestovali 1. tridou + u lidi, kteri prezili, takze si vytvorim novy sloupec _cabin_info_ , ktery bude zastupovat zda je udaj o kabine dostupny, ci nikoli
    * po ozkouseni mi to vysledne hodnoty nijak nezlepsovalo, po _x_ pokusech mi prislo, ze to naopak zhorsovalo vysledky, a proto jsem se rozhodl tuto informaci nevyuzit

In [5]:
# data['cabin_info'] = data['cabin'].notna()

* pokus o rozliseni jednotlivych kabin dle podlazi (prvni pismeno z nazvu cabiny), kde jsem nedostupnym udajum priradil priznak _N_
    * po aplikaci jsem zaznamenal vyrazne zhorseni vysledku -> takze jsem ve finale take nevyuzil

In [6]:
# for i, row in data.loc[data.cabin.notnull()].iterrows():
#     data.at[i, 'cabin'] = row['cabin'][slice(1)]
    
# for i, row in data.loc[data.cabin.isnull()].iterrows():
#     data.at[i, 'cabin'] = 'N'

* Dropnu sloupce, ktere jsou nerelevantni pro predpoved nasich dat
    * ticket -> cislo lodniho listku podle me neni relevantni vuci tomu kdo prezil
    * name -> jmena take
    * home.dest -> take nejspise nemelo vliv a velke mnozstvi chybejicich hodnot me vedlo k rozhodnuti tento sloupec dropnout
    * cabin -> chybi 773 z 1000 dat a i pri pokusech vytezit z dostupnych informaci nejakeho zlepseni sem nedospel k uspesnemu zaveru, proto i tento sloupec vyrazuji
    * ID -> ID cestujiciho v datasetu take neni relevantni info, z ktereho bychom meli vychazet

In [7]:
data.drop(columns=['ticket', 'name', 'home.dest', 'cabin', 'ID'], inplace=True)

* Kontrola, ze nyni jiz pouze sloupec **age** obsahuje chybejici data, ta doplnim v nasledujici casti

In [8]:
print('Missing data in columns:')
print(data.loc[:,data.isnull().sum() > 0].isnull().sum())

Missing data in columns:
age    201
dtype: int64


* Nejdrive zmenim textove priznaky na kategorialni (jedna se o **sex** a **embarked** sloupce)

In [9]:
string_cols = data.select_dtypes('object').columns
# display(data[string_cols].nunique())
data[string_cols] = data[string_cols].astype('category').apply(lambda x: x.cat.codes)
# data.info()

Napadly me dva zpusoby jak doplnit **age** hodnotu, tam kde chybi
* pomoci medianu
* pomoci kNN regressoru
Po ozkouseni obou variant sem k vyrazne lepsim vysledkum dochazel u pouziti prvni zminene metody (az o 10% zlepseni na testovacich datech), proto je pouziti kNN regressoru nize zakomentovano

In [10]:
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
data[['age']] = imputer.fit_transform(data[['age']])

Pro kNN regressor nejdrive data rozdelim na 3 casti (trenovaci, validacni a testovaci) a pak na vyplnenych **age** sloupcich v trenovacich datech naucim kNN regressor predpovidat hodnotu veku, tu pak pripadne pouziji na zbytek trenovacich dat, data validacni a data testovaci

In [11]:
randomSeed = 21; # just so I fix it and always get the same results

# split data to training, validation and test sets
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(data.drop(columns = ['survived']), data['survived'], test_size=0.25, random_state=randomSeed)
Xtrain, Xvalid, Ytrain, Yvalid = train_test_split(Xtrain, Ytrain, test_size=0.25, random_state=randomSeed)

In [12]:
# import warnings
# warnings.filterwarnings('ignore')
# from sklearn.neighbors import KNeighborsRegressor

# XtrainWithAge = Xtrain[Xtrain['age'].notnull()]
# XtrainNoAge = Xtrain[Xtrain['age'].isnull()]

# # divide other data sets to "withAgeFilled" and "missingAgeValue"
# XvalidWithAge = Xvalid[Xvalid['age'].notnull()]
# XvalidNoAge = Xvalid[Xvalid['age'].isnull()]
# XtestWithAge = Xtest[Xtest['age'].notnull()]
# XtestNoAge = Xtest[Xtest['age'].isnull()]

# # train on filled values in training data
# XtrainAge = XtrainWithAge.drop(columns = ['age'])
# YtrainAge = XtrainWithAge['age']

# # data to fill
# XtrainPredictAge = XtrainNoAge.drop(columns = ['age'])
# XvalidPredictAge = XvalidNoAge.drop(columns = ['age'])
# XtestPredictAge = XtestNoAge.drop(columns = ['age'])

# # train
# knn = KNeighborsRegressor(n_neighbors=5)
# knn.fit(XtrainAge,YtrainAge)

# # predict values
# YtrainPredictAge = knn.predict(XtrainPredictAge)
# YvalidPredictAge = knn.predict(XvalidPredictAge)
# YtestPredictAge = knn.predict(XtestPredictAge)

# # fill values
# XtrainNoAge['age'] = YtrainPredictAge
# XvalidNoAge['age'] = YvalidPredictAge
# XtestNoAge['age'] = YtestPredictAge

# # concat data
# Xtrain = pd.concat([XtrainWithAge, XtrainNoAge])
# Xvalid = pd.concat([XvalidWithAge, XvalidNoAge])
# Xtest = pd.concat([XtestWithAge, XtestNoAge])

Kontrola zda jsou jiz vsechny sloupce vyplnene

In [13]:
# print(Xtrain.isnull().sum() == 0)
# print(Xvalid.isnull().sum() == 0)
# print(Xtest.isnull().sum() == 0)

Vytvorena a pripravena data jiz menit nebudu, budu si vzdy vytvaret pro kazdy model jejich kopie a nad temi dane modely a predikce spoustet

In [14]:
def copyComputedData():
    XtrainCopy = Xtrain.copy()
    YtrainCopy = Ytrain.copy() 
    XvalidCopy = Xvalid.copy() 
    YvalidCopy = Yvalid.copy() 
    XtestCopy = Xtest.copy() 
    YtestCopy = Ytest.copy()
    return XtrainCopy, YtrainCopy, XvalidCopy, YvalidCopy, XtestCopy, YtestCopy

**DECISION TREE**

In [15]:
XtrainCopy, YtrainCopy, XvalidCopy, YvalidCopy, XtestCopy, YtestCopy = copyComputedData()

from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics

# build the tree and find the optimal depth of the tree
validationAccuracyScoreList = []
for depth in range(2,60):
    dtc = DecisionTreeClassifier(max_depth=depth, random_state=randomSeed)
    dtc.fit(XtrainCopy, YtrainCopy)
    validationAccuracyScoreList.append(metrics.accuracy_score(YvalidCopy, dtc.predict(XvalidCopy)))

optimalDepth = np.argmax(validationAccuracyScoreList) + 1

# predict values with the optimal depth
dtc = DecisionTreeClassifier(max_depth=optimalDepth, random_state=randomSeed)
dtc.fit(XtrainCopy, YtrainCopy)

YtrainPrediction = dtc.predict(XtrainCopy)
YvalidPrediction = dtc.predict(XvalidCopy)
YtestPrediction = dtc.predict(XtestCopy)

print('Accuracy on train set: {0:.6f}'.format(metrics.accuracy_score(YtrainCopy, YtrainPrediction)))
print('Accuracy on validation set: {0:.6f}'.format(metrics.accuracy_score(YvalidCopy, YvalidPrediction)))
print('Accuracy on test set: {0:.6f}'.format(metrics.accuracy_score(YtestCopy, YtestPrediction)))

Accuracy on train set: 0.877224
Accuracy on validation set: 0.787234
Accuracy on test set: 0.812000


**Bagging**

In [16]:
XtrainCopy, YtrainCopy, XvalidCopy, YvalidCopy, XtestCopy, YtestCopy = copyComputedData()

import sklearn.metrics as metrics
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier

# search these ranges of parameters to find which parameter produces the best result
paramGrid = {
    'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 70, 100, 150, 190],
    'max_depth': range(3,42)
}
paramCombinations = ParameterGrid(paramGrid)

# train the RandomForestClassifier for each combination of parameters
validationAccuracyScoreList = []
for params in paramCombinations:
    rfc = RandomForestClassifier(**params, random_state=randomSeed)
    rfc.fit(XtrainCopy, YtrainCopy)
    validationAccuracyScoreList.append(metrics.accuracy_score(YvalidCopy, rfc.predict(XvalidCopy)))

# find which parameters give the best result
bestParamsBagging = paramCombinations[np.argmax(validationAccuracyScoreList)]

#train the model with the best parameters
rfc = RandomForestClassifier(**bestParamsBagging, random_state=randomSeed)
rfc.fit(XtrainCopy, YtrainCopy)

YtrainPrediction = rfc.predict(XtrainCopy)
YvalidPrediction = rfc.predict(XvalidCopy)
YtestPrediction = rfc.predict(XtestCopy)

print('Accuracy of Bagging method')
print('Accuracy on train set: {0:.6f}'.format(metrics.accuracy_score(YtrainCopy, YtrainPrediction)))
print('Accuracy on validation set: {0:.6f}'.format(metrics.accuracy_score(YvalidCopy, YvalidPrediction)))
print('Accuracy on test set: {0:.6f}'.format(metrics.accuracy_score(YtestCopy, YtestPrediction)))

Accuracy of Bagging method
Accuracy on train set: 0.882562
Accuracy on validation set: 0.813830
Accuracy on test set: 0.800000


**Adaboost**

In [17]:
XtrainCopy, YtrainCopy, XvalidCopy, YvalidCopy, XtestCopy, YtestCopy = copyComputedData()

from sklearn.ensemble import AdaBoostClassifier

# try these parameters to find which parameter produces the best result
paramGrid = {
    'n_estimators': range(1,100,5),
    'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.075, 0.1, 0.2, 0.3, 0.4, 0.5, 0.75, 1]
}
paramCombinations = ParameterGrid(paramGrid)

# train the AdaBoostClassifier for each combination of parameters
validationAccuracyScoreList = []
for params in paramCombinations:
    abc = AdaBoostClassifier(**params, random_state=randomSeed)
    abc.fit(XtrainCopy, YtrainCopy)
    validationAccuracyScoreList.append(metrics.accuracy_score(YvalidCopy, abc.predict(XvalidCopy)))

# find which parameters give the best result
bestParamsAdaBoost = paramCombinations[np.argmax(validationAccuracyScoreList)]

#train the model with the best parameters
abc = AdaBoostClassifier(**bestParamsAdaBoost, random_state=randomSeed)
abc.fit(XtrainCopy, YtrainCopy)

YtrainPrediction = abc.predict(XtrainCopy)
YvalidPrediction = abc.predict(XvalidCopy)
YtestPrediction = abc.predict(XtestCopy)

print('Accuracy of AdaBoost method')
print('Accuracy on train set: {0:.6f}'.format(metrics.accuracy_score(YtrainCopy, YtrainPrediction)))
print('Accuracy on validation set: {0:.6f}'.format(metrics.accuracy_score(YvalidCopy, YvalidPrediction)))
print('Accuracy on test set: {0:.6f}'.format(metrics.accuracy_score(YtestCopy, YtestPrediction)))

Accuracy of AdaBoost method
Accuracy on train set: 0.811388
Accuracy on validation set: 0.781915
Accuracy on test set: 0.804000


**kNN**
* pouzil jsem zakladni minkowskeho metriku
* pri znormalizovani hodnot jsem dosahl zlepseni vysledku

In [18]:
XtrainCopy, YtrainCopy, XvalidCopy, YvalidCopy, XtestCopy, YtestCopy = copyComputedData()

from sklearn.neighbors import KNeighborsClassifier
import math
    
# try these parameters to find which parameter produces the best result
paramGrid = {
    'n_neighbors' : range(2,21),
    'p': [1,2],
    'weights': ['uniform', 'distance']
}
paramCombinations = ParameterGrid(paramGrid)

# normalize the values
XtrainNormalized = (XtrainCopy - XtrainCopy.min(axis=0))/(XtrainCopy.max(axis = 0) - XtrainCopy.min(axis = 0))
XvalidNormalized = (XvalidCopy - XvalidCopy.min(axis=0))/(XvalidCopy.max(axis = 0) - XvalidCopy.min(axis = 0))
XtestNormalized = (XtestCopy - XtestCopy.min(axis=0))/(XtestCopy.max(axis = 0) - XtestCopy.min(axis = 0))

# train the kNN for each combination of parameters
validationAccuracyScoreList = []
for params in paramCombinations:
    kNN = KNeighborsClassifier(**params)
    kNN.fit(XtrainNormalized, YtrainCopy)
    validationAccuracyScoreList.append(metrics.accuracy_score(YvalidCopy, kNN.predict(XvalidNormalized)))
    
# find which parameters give the best result
bestParamsKNN = paramCombinations[np.argmax(validationAccuracyScoreList)]

#train the model with the best parameters
kNN = KNeighborsClassifier(**bestParamsKNN)
kNN.fit(XtrainNormalized, YtrainCopy)

YtrainPrediction = kNN.predict(XtrainNormalized)
YvalidPrediction = kNN.predict(XvalidNormalized)
YtestPrediction = kNN.predict(XtestNormalized)

print('Accuracy of kNN classifier method')
print('Accuracy on train set: {0:.6f}'.format(metrics.accuracy_score(YtrainCopy, YtrainPrediction)))
print('Accuracy on validation set: {0:.6f}'.format(metrics.accuracy_score(YvalidCopy, YvalidPrediction)))
print('Accuracy on test set: {0:.6f}'.format(metrics.accuracy_score(YtestCopy, YtestPrediction)))

Accuracy of kNN classifier method
Accuracy on train set: 0.982206
Accuracy on validation set: 0.797872
Accuracy on test set: 0.784000


Nakonec jsem se rozhodl pro vyhodnoceni _evaluation_ datasetu pouzit **Adaboost**
* v _evaluation.csv_ take chybeli pouze data ze 3 sloupcu, zachazel jsem s nimi tedy stejnym zpusobem jako s trenovacimi daty

In [19]:
evaluationData = pd.read_csv('evaluation.csv')
# display(result.info())
# result.loc[:,result.isnull().sum() > 0].isnull().sum()

In [20]:
# extract ID's
ids = evaluationData['ID'].values

# preprocess data
# drop columns
evaluationData.drop(columns=['ticket', 'name', 'home.dest', 'cabin', 'ID'], inplace=True)
# fill age values
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
evaluationData[['age']] = imputer.fit_transform(evaluationData[['age']])
# change text variables to categorial
string_cols = evaluationData.select_dtypes('object').columns
evaluationData[string_cols] = evaluationData[string_cols].astype('category').apply(lambda x: x.cat.codes)

# predict by the trained AdaBoost from above
YevaluationDatePrediction = abc.predict(evaluationData)

evaluationData['ID'] = ids
evaluationData['survived'] = YevaluationDatePrediction
evaluationData.drop(columns=['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked'], inplace=True)

evaluationData.to_csv('results.csv')
evaluationData.head()

,ID,survived
0,1000,0
1,1001,0
2,1002,0
3,1003,0
4,1004,0
